# ****============ CUSTOMER SEGMENTATION =============****

En este cuaderno, exportaré datos e intentaré segmentar a los clientes en un mercado de Brasil utilizando el método de análisis RFM. El objetivo de este proyecto es ayudar al equipo de marketing a determinar su mercado objetivo para ser más específico. Dividiré este cuaderno en las siguientes etapas:

    Contexto
    Preprocesamiento de datos
    Modelado
    Conclusión

Este cuaderno no es demasiado profundo para discutir la teoría, pero daré razones por las cuales hice estos pasos. ¡Empecemos!
1. CONTEXTO

Este conjunto de datos es Olist, el mercado más grande de Brasil. Esta información se recopiló en el período 2016-2018. Olist conecta a pequeñas empresas de todo Brasil para ingresar a sus redes de manera muy sencilla simplemente mediante un contrato. Estos empresarios pueden vender sus productos a través de la plataforma Olist y enviarlos directamente a sus clientes con correos que han trabajado igual que Olist.

Después de que el cliente compre su producto a Olist, el vendedor recibirá una notificación para cumplir de inmediato con el pedido del cliente. Cuando un cliente recibe un producto, el cliente puede dar una calificación de satisfacción y comentarios relacionados con los servicios prestados por el vendedor a través del correo electrónico enviado.

Los datos recopilados están en 9 archivos diferentes, por lo que deben combinarse para obtener todos los datos en función de los parámetros que deseamos. A continuación se muestra un esquema de conexión entre los datos que se utilizarán como punto de referencia al fusionar datos.

![](https://i.imgur.com/HRhd2Y0.png)

In [ ]:
!pip install yellowbrick

In [1]:
#Importing Libraries
#Basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

#Fetaure Selection
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
#Modelling Algoritm
from sklearn.cluster import KMeans

#Model Evaluation
from yellowbrick.cluster import SilhouetteVisualizer

/home/bandrea/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#Load All The Data
olist_orders = pd.read_csv('./olist_orders_dataset.csv')
olist_products = pd.read_csv('./olist_products_dataset.csv')
olist_items = pd.read_csv('./olist_order_items_dataset.csv')
olist_customers = pd.read_csv('./olist_customers_dataset.csv')
olist_payments = pd.read_csv('./olist_order_payments_dataset.csv')
olist_sellers = pd.read_csv('./olist_sellers_dataset.csv')
olist_geolocation = pd.read_csv('./olist_geolocation_dataset.csv')
olist_reviews = pd.read_csv('./olist_order_reviews_dataset.csv')
olist_product_category_name = pd.read_csv('./product_category_name_translation.csv')

In [3]:
olist_product_category_name.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [4]:
all_data = olist_orders.merge(olist_items, on='order_id', how='left')
all_data = all_data.merge(olist_payments, on='order_id', how='inner')
all_data = all_data.merge(olist_reviews, on='order_id', how='inner')
all_data = all_data.merge(olist_products, on='product_id', how='inner')
all_data = all_data.merge(olist_customers, on='customer_id', how='inner')
all_data = all_data.merge(olist_sellers, on='seller_id', how='inner')
all_data = all_data.merge(olist_product_category_name,on='product_category_name',how='inner')
#all_data = all_data.merge(olist_geolocation, on='seller_zip_code_prefix', how='inner')

In [5]:
#Vea qué porcentaje de datos está en blanco en cada columna
round((all_data.isnull().sum()/ len(all_data)*100),2)

order_id                          0.00
customer_id                       0.00
order_status                      0.00
order_purchase_timestamp          0.00
order_approved_at                 0.01
order_delivered_carrier_date      1.04
order_delivered_customer_date     2.16
order_estimated_delivery_date     0.00
order_item_id                     0.00
product_id                        0.00
seller_id                         0.00
shipping_limit_date               0.00
price                             0.00
freight_value                     0.00
payment_sequential                0.00
payment_type                      0.00
payment_installments              0.00
payment_value                     0.00
review_id                         0.00
review_score                      0.00
review_comment_title             87.99
review_comment_message           57.22
review_creation_date              0.00
review_answer_timestamp           0.00
product_category_name             0.00
product_name_lenght      

In [6]:
# Viendo la información en los datos tanto el número de columnas, como la entrada a la memoria
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116581 entries, 0 to 116580
Data columns (total 40 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       116581 non-null  object 
 1   customer_id                    116581 non-null  object 
 2   order_status                   116581 non-null  object 
 3   order_purchase_timestamp       116581 non-null  object 
 4   order_approved_at              116567 non-null  object 
 5   order_delivered_carrier_date   115368 non-null  object 
 6   order_delivered_customer_date  114066 non-null  object 
 7   order_estimated_delivery_date  116581 non-null  object 
 8   order_item_id                  116581 non-null  float64
 9   product_id                     116581 non-null  object 
 10  seller_id                      116581 non-null  object 
 11  shipping_limit_date            116581 non-null  object 
 12  price                         

In [ ]:
all_data.to_csv('./csegmentation.csv')

# ** 2. PREPROCESAMIENTO DE DATOS **

En esta etapa, dejaremos los datos fuera del ruido / ruido (entradas en blanco, valores atípicos, etc.) para que los datos que se ingresen en el modelo estén limpios y listos para el modelado.

## ** 2.1 Manejo variable **

Esta etapa tiene como objetivo ajustar los tipos de datos en función de su entrada y también eliminar la duplicación de los datos que tenemos



In [7]:
from preprocesamiento import preprocesa
all_data2= preprocesa(all_data)

                                order_id                       customer_id  \
0       e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1       e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
2       e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
3       128e10d95713541c87cd1a2e48201934  a20e8105f23924cd00833fd87daa0831   
4       0e7e841ddf8f8f2de2bad69267ecfbcf  26c7ac168e1433912a51b924fbd34d34   
...                                  ...                               ...   
116576  442a41b00a8a8bda35511fdf1eb65cc5  e42e2e8642f9c09100b6667e321c2811   
116577  1dbff949801376b8795a093d31cfe205  0cbd2df3d6e9d8b1eaf5cfefc44886bc   
116578  6775b950cdc4da0ce39b261032f92c84  c4d135422be0d4333f506acde8af9419   
116579  73d60420cd1a179b2d8887d538efe4c2  65d9213dfa004c17dc126503e106e4a3   
116580  f5cf5716413185387030a378bdd46ebe  ff95df9a387c8d032a2ca6887cf77236   

       order_status order_purchase_timestamp    order_approved_

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing

In [ ]:
all_data['order_purchase_timestamp'].dtype

In [ ]:
all_data2.columns

In [ ]:
# Cambie el tipo de datos en la columna de fecha para que el tipo de datos finalice
date_columns = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date',
             'order_estimated_delivery_date', 'shipping_limit_date', 'review_creation_date', 'review_answer_timestamp'] 
#date_columns = ['order_delivered_carrier_date']
for col in date_columns:
    all_data[col] = pd.to_datetime(all_data[col], format='%Y-%m-%d %H:%M:%S')

In [ ]:
all_data['order_delivered_carrier_date']

In [ ]:
# Ver si hay datos duplicados
print('Duplicados: ',all_data.duplicated().sum())

In [ ]:
#Cree una columna de mes_orden para la exploración de datos
all_data['Month_order'] = all_data['order_purchase_timestamp'].dt.to_period('M').astype('str')

In [ ]:
all_data[['Month_order','order_purchase_timestamp']].head()

In [ ]:
# Elija entradas que van desde 01-2017 hasta 08-2018
#Porque hay datos que están fuera de balance con el promedio de cada mes en los datos antes del 01-2017 y después del 08-2018
# basado en datos de compra / order_purchase_timestamp
start_date = "2017-01-01"
end_date = "2018-08-31"

after_start_date = all_data['order_purchase_timestamp'] >= start_date
before_end_date = all_data['order_purchase_timestamp'] <= end_date
between_two_dates = after_start_date & before_end_date
all_data = all_data.loc[between_two_dates]

In [ ]:
# Compartir datos según el tipo de datos
only_numeric = all_data.select_dtypes(include=['int', 'float'])
only_object = all_data.select_dtypes(include=['object'])
only_time = all_data.select_dtypes(include=['datetime', 'timedelta'])

## 2.2 Manejo de valores perdidos

En esta etapa, se hace para eliminar entradas vacías mediante el uso de otras características o el uso de estadísticas (media / mediana)

In [ ]:
# Vea qué porcentaje de datos está en blanco en cada columna
round((all_data.isnull().sum()/ len(all_data)*100),2)

In [ ]:
# Gestiona entradas vacías en la columna order_approved_at
missing_1 = all_data['order_approved_at'] - all_data['order_purchase_timestamp']
print(missing_1.describe())
print('='*50)
print('Mediana desde el momento en que se aprobó la orden: ',missing_1.median())

# tomamos la mediana porque hay quienes aprueban directamente desde el momento en que ordena, algunos son de hasta 60 días
add_1 = all_data[all_data['order_approved_at'].isnull()]['order_purchase_timestamp'] + missing_1.median()
all_data['order_approved_at']= all_data['order_approved_at'].replace(np.nan, add_1)

In [ ]:
# Gestiona entradas vacías en la columna order_approved_at
all_data[['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']].head()

In [ ]:
# Gestión de entradas vacías en la columna order_delivered_carrier_date
missing_2 = all_data['order_delivered_carrier_date'] - all_data['order_approved_at']
print(missing_2.describe())
print('='*50)
print('Mediana desde el momento de la solicitud hasta el envío: ',missing_2.median())

# Tomamos la mediana porque algunos barcos están dentro de las 21 horas del tiempo acordado, algunos hasta 107 días
add_2 = all_data[all_data['order_delivered_carrier_date'].isnull()]['order_approved_at'] + missing_2.median()
all_data['order_delivered_carrier_date']= all_data['order_delivered_carrier_date'].replace(np.nan, add_2)

In [ ]:
# Gestión de entradas vacías en la columna order_delivered_customer_date
missing_3 = all_data['order_delivered_customer_date'] - all_data['order_delivered_carrier_date']
print(missing_3.describe())
print('='*50)
print('Mediana desde el momento en que se envió hasta que el cliente la recibió: ',missing_3.median())

# tomamos la mediana porque hay un tiempo de entrega de -17 días, lo que significa que es atípico, también hay un tiempo de entrega de hasta 205 días
add_3 = all_data[all_data['order_delivered_customer_date'].isnull()]['order_delivered_carrier_date'] + missing_3.median()
all_data['order_delivered_customer_date']= all_data['order_delivered_customer_date'].replace(np.nan, add_3)

In [ ]:
# Manejar las columnas review_comment_title y review_comment_message
#Porque el número de entradas en blanco es muy grande e imposible de completar porque no hay variables que puedan
# usado para calcularlo. Porque este es el comentario y el título del comentario
# Luego eliminaremos la columna

all_data = all_data.drop(['review_comment_title', 'review_comment_message'], axis=1)

In [ ]:
# Entrega de entrada vacía en las columnas product_weight_g, product_length_cm, product_height_cm, product_width_cm
#Porque solo hay 1, entonces lo dejamos caer
all_data = all_data.dropna()

In [ ]:
# Compruebe si hay entradas en blanco
round((all_data.isnull().sum()/len(all_data)*100),2)

In [ ]:
# Ajuste el tipo de datos con los datos de entrada
all_data = all_data.astype({'order_item_id': 'int64', 
                            'product_name_lenght': 'int64',
                            'product_description_lenght':'int64', 
                            'product_photos_qty':'int64'})

## ** 2.3. Extracción de características **

En esta etapa, la adición de nuevas columnas que contienen cálculos de varias columnas para obtener una nueva característica



In [ ]:
#Cree una columna order_process_time para ver cuánto tiempo llevará iniciar el pedido hasta
# artículos son aceptados por los clientes
all_data['order_process_time'] = all_data['order_delivered_customer_date'] - all_data['order_purchase_timestamp']

In [ ]:
#Cree una columna order_delivery_time para ver cuánto tiempo se requiere el tiempo de envío para cada pedido
all_data['order_delivery_time'] = all_data['order_delivered_customer_date'] - all_data['order_delivered_carrier_date']

In [ ]:
#Cree una columna order_time_accuracy para ver si desde el tiempo estimado hasta que algo sea apropiado o tarde
# Si el valor es + positivo, entonces es más rápido hasta que, si es 0, está justo a tiempo, pero si es negativo, llega tarde
all_data['order_accuracy_time'] = all_data['order_estimated_delivery_date'] - all_data['order_delivered_customer_date'] 

In [ ]:
#Cree una columna order_approved_time para ver cuánto tiempo tomará desde el pedido hasta la aprobación
all_data['order_approved_time'] = all_data['order_approved_at'] - all_data['order_purchase_timestamp'] 

In [ ]:
#Cree una columna review_send_time para averiguar cuánto tiempo se envió la encuesta de satisfacción después de recibir el artículo.
all_data['review_send_time'] = all_data['review_creation_date'] - all_data['order_delivered_customer_date']

In [ ]:
#Cree una columna review_answer_time para averiguar cuánto tiempo llevará completar una revisión después de
# envió una encuesta de satisfacción del cliente.
all_data['review_answer_time'] = all_data['review_answer_timestamp'] - all_data['review_creation_date']

In [ ]:
# Combine las columnas product_length_cm, product_height_cm y product_width_cm para convertirlo en un volumen
# con una nueva columna, volumen_producto
all_data['product_volume'] = all_data['product_length_cm'] * all_data['product_height_cm'] * all_data['product_width_cm']

In [ ]:
all_data['product_volume'].nunique()

In [ ]:
all_data['order_process_time'].mean()

## ** 2.4 Exploración de datos **

Esta etapa se lleva a cabo la exploración de datos para obtener información o información que puede ser útil para la empresa.

In [ ]:
# ¿Qué productos tienen más demanda?
top_20_product_best_seller = all_data['order_item_id'].groupby(all_data['product_category_name_english']).sum().sort_values(ascending=False)[:20]
#print(top_20_product_best_seller)

# Lo trazamos para visualización
fig=plt.figure(figsize=(16,9))
sns.barplot(y=top_20_product_best_seller.index,x=top_20_product_best_seller.values)
plt.title('Top 20 Most Selling Product',fontsize=20)
plt.xlabel('Total Product Sold',fontsize=17)
plt.ylabel('Product category',fontsize=17)

In [ ]:
top_20_city_shopping = all_data['order_item_id'].groupby(all_data['customer_city']).sum().sort_values(ascending=False)[:20]
#print(top_20_city_shopping)

# ¿Qué ciudad compra más?
fig=plt.figure(figsize=(16,9))
sns.barplot(y=top_20_city_shopping.index,x=top_20_city_shopping.values)
plt.title('Top 20 Most City Shopping',fontsize=20)
plt.xlabel('Total Product',fontsize=17)
plt.ylabel('City',fontsize=17)

In [ ]:
# ¿Quién es el mayor número de compras de clientes en función de la cantidad de pedidos?
top_10_customer_shopping = all_data['order_item_id'].groupby(all_data['customer_id']).count().sort_values(ascending=False)[:10]
#print(top_10_customer_shopping)

# Lo trazamos para visualización
fig=plt.figure(figsize=(16,9))
sns.barplot(y=top_10_customer_shopping.index,x=top_10_customer_shopping.values)
plt.title('Top 10 Customer Based on Order Amount',fontsize=20)
plt.xlabel('Amount of Product',fontsize=17)
plt.ylabel('Customer ID',fontsize=17)

In [ ]:
# ¿Quién es el cliente con más gasto en compras por precio?
top_10_customer_shopping = all_data['payment_value'].groupby(all_data['customer_id']).sum().sort_values(ascending=False)[:10]
#print(top_10_customer_shopping)

fig=plt.figure(figsize=(16,9))
sns.barplot(y=top_10_customer_shopping.index,x=top_10_customer_shopping.values)
plt.title('Top 10 Customer Based on Spending',fontsize=20)
plt.xlabel('Spending Amount',fontsize=17)
plt.ylabel('Customer ID',fontsize=17)

In [ ]:
# ¿Qué vendedores venden más?
top_10_seller_order = all_data['order_item_id'].groupby(all_data['seller_id']).sum().sort_values(ascending=False)[:10]
#print(top_10_seller_order)

fig=plt.figure(figsize=(16,9))
sns.barplot(y=top_10_seller_order.index,x=top_10_seller_order.values)
plt.title('Top 10 Seller Base on Sold Product',fontsize=20)
plt.xlabel('Total Product',fontsize=17)
plt.ylabel('Seller ID',fontsize=17)

In [ ]:
# Vendedor, ¿cuál es el mayor ingreso basado en ingresos?
top_10_seller_order = all_data['price'].groupby(all_data['seller_id']).sum().sort_values(ascending=False)[:10]
#print(top_10_seller_order)

fig=plt.figure(figsize=(16,9))
sns.barplot(y=top_10_seller_order.index,x=top_10_seller_order.values)
plt.title('Top 10 Seller Based on Revenue',fontsize=20)
plt.xlabel('Amount of Revenue',fontsize=17)
plt.ylabel('Seller ID',fontsize=17)

In [ ]:
# Vendedor, ¿cuál es el mayor ingreso basado en ingresos?
top_10_seller_order = all_data[all_data['review_score'] == 5].groupby(all_data['seller_id']).sum().sort_values(by=['review_score'],ascending=False)[:10]
#print(top_10_seller_order)

fig=plt.figure(figsize=(16,9))
sns.barplot(y=top_10_seller_order.index,x=top_10_seller_order.review_score)
plt.title('Top 10 Seller Based on Review Score',fontsize=20)
plt.xlabel('Amount of Revenue',fontsize=17)
plt.ylabel('Seller ID',fontsize=17)

In [ ]:
# Distribución del estado del pedido del cliente
round(all_data.order_status.value_counts() / len(all_data),2)

In [ ]:
# ¿Cuál es el tiempo promedio desde el pedido hasta el recibo que se necesita en cada pedido mensual?
order_time_by_month = all_data['order_process_time'].groupby(all_data['Month_order']).median(numeric_only=False) #masukan argumen numeric_only untuk menghitung timedelta

fig=plt.figure(figsize=(16,9))
plt.plot(order_time_by_month.index, order_time_by_month.values, marker='o')
plt.title('Median Order Time By Month',fontsize=20)
plt.xlabel('Month',fontsize=17)
plt.xticks(#[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
          rotation=90)
plt.ylabel('Time (Day)',fontsize=17)

In [ ]:
# ¿Cuál es el tiempo de entrega promedio requerido para cada primer pedido?
delivery_time_by_month = all_data['order_delivery_time'].groupby(all_data['Month_order']).median(numeric_only=False) #masukan argumen numeric_only untuk menghitung timedelta


fig=plt.figure(figsize=(16,9))
plt.plot(delivery_time_by_month.index, delivery_time_by_month.values / 86400, marker='o')
plt.title('Median Delivery Time By Month',fontsize=20)
plt.xlabel('Month',fontsize=17)
plt.xticks(#[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
          rotation=90)
plt.ylabel('Time (Day)',fontsize=17)

In [ ]:
# ¿Cuál es la precisión media del tiempo de los envíos estimados y hasta el cliente en cada pedido mensual?
accuracy_time_by_month = all_data['order_accuracy_time'].groupby(all_data['Month_order']).median(numeric_only=False) #masukan argumen numeric_only untuk menghitung timedelta

fig=plt.figure(figsize=(16,9))
plt.plot(accuracy_time_by_month.index, accuracy_time_by_month.values / 86400, marker='o')
plt.title('Median Accuracy Time By Month',fontsize=20)
plt.xlabel('Month',fontsize=17)
plt.xticks(#[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
          rotation=90)
plt.ylabel('Time (Day)',fontsize=17)

In [ ]:
# ¿Cuál es el período de tiempo promedio hasta que se aprueba desde el momento del pedido en cada pedido mensual?
approved_time_by_month = all_data['order_approved_time'].groupby(all_data['Month_order']).median(numeric_only=False) #masukan argumen numeric_only untuk menghitung timedelta

fig=plt.figure(figsize=(16,9))
plt.plot(approved_time_by_month.index, approved_time_by_month.values / 60, marker='o')
plt.title('Median Approved Time By Month',fontsize=20)
plt.xlabel('Month',fontsize=17)
plt.xticks(#[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
          rotation=90)
plt.ylabel('Time (Minutes)',fontsize=17)

In [ ]:
# 10 categorías de productos con el tiempo más rápido desde el pedido hasta la aceptación del cliente
order_time_by_category = pd.DataFrame(all_data['order_process_time'].groupby(all_data['product_category_name_english']).median(numeric_only=False).sort_values(ascending=True)[:10])

fig=plt.figure(figsize=(16,9))
sns.barplot(y=order_time_by_category.index, x=order_time_by_category['order_process_time'].dt.days)
plt.title('Top 10 Fastest Product Category Order Time',fontsize=20)
plt.xlabel('Order Time (Day)',fontsize=17)
plt.ylabel('Product Category',fontsize=17)

In [ ]:
# 10 categorías de productos con el mayor tiempo desde el pedido hasta la aceptación del cliente
order_time_by_category = pd.DataFrame(all_data['order_process_time'].groupby(all_data['product_category_name_english']).median(numeric_only=False).sort_values(ascending=False)[:10])

fig=plt.figure(figsize=(16,9))
sns.barplot(y=order_time_by_category.index, x=order_time_by_category['order_process_time'].dt.days)
plt.title('Top 10 Slowest Product Category Order Time',fontsize=20)
plt.xlabel('Order Time (Day)',fontsize=17)
plt.ylabel('Product Category',fontsize=17)

In [ ]:
# ¿Cuánto cuesta el pedido cada mes?
order_count_by_month = all_data['order_item_id'].groupby(all_data['Month_order']).sum()

fig=plt.figure(figsize=(16,9))
sns.barplot(y=order_count_by_month.values, x=order_count_by_month.index, color="Salmon")
plt.title('Monthly Order',fontsize=20)
plt.xlabel('Month',fontsize=17)
plt.xticks(rotation=90)
plt.ylabel('Amount Order',fontsize=17)

In [ ]:
# ¿Cuánto es el ingreso mensual?
revenue_count_by_month = all_data['payment_value'].groupby(all_data['Month_order']).sum()

fig=plt.figure(figsize=(16,9))
sns.barplot(y=revenue_count_by_month.values, x=revenue_count_by_month.index, color="Salmon")
plt.title('Monthly Revenue',fontsize=20)
plt.xlabel('Month',fontsize=17)
plt.xticks(rotation=90)
plt.ylabel('Amount Revenue',fontsize=17)

In [ ]:
# ¿Cómo son los clientes activos cada mes?
customer_active_by_month = all_data.groupby('Month_order')['customer_unique_id'].nunique().reset_index()

fig=plt.figure(figsize=(16,9))
sns.barplot(y=customer_active_by_month['customer_unique_id'], x=customer_active_by_month['Month_order'], color="Salmon")
plt.title('Monthly Active User',fontsize=20)
plt.xlabel('Month',fontsize=17)
plt.xticks(rotation=90)
plt.ylabel('Amount of User',fontsize=17)

# ** 3. Modelado **

En esta etapa, el modelado se llevará a cabo utilizando un algoritmo de agrupamiento denominado K-Means basado en el análisis RFM (Recency, Frequency, and Monetary) para la segmentación de clientes.

## ** 3.1 Análisis RFM **

El análisis RFM es un método utilizado para ver patrones en los clientes en función de sus hábitos de compra:

1. Recency = ¿Cuándo fue la última vez que el cliente realizó una transacción?
 
2. Frecuencia = ¿Con qué frecuencia el cliente realiza una compra?

3. Monetario = cuánto gastan los clientes

In [ ]:
#ver las fechas de compra e inicio
print('Min : {}, Max : {}'.format(min(all_data.order_purchase_timestamp), max(all_data.order_purchase_timestamp)))

In [ ]:
pin_date


In [ ]:
pin_date - all_data.groupby('customer_unique_id')['order_purchase_timestamp'].max().iloc[1]

In [ ]:
#Calcular RFM
import datetime as dt
pin_date = max(all_data.order_purchase_timestamp) + dt.timedelta(1)

#Creo un dataframe para RFM
rfm = all_data.groupby('customer_unique_id').agg({
    'order_purchase_timestamp' : lambda x: (pin_date - x.max()).days,
    'order_item_id' : 'count', 
    'payment_value' : 'sum'})

In [ ]:
rfm.head()

In [ ]:
rfm.rename(columns = {'order_purchase_timestamp' : 'Recency', 
                      'order_item_id' : 'Frequency', 
                      'payment_value' : 'Monetary'}, inplace = True)

rfm.head(10)

## ** 3.2 Manejo de valores atípicos **

En esta etapa, limpiaremos los datos de las entradas que pueden ser incorrectas o las anomalías de los datos que tenemos.

In [ ]:
#Usamos Rangos intercuartilicos
def limit(i):
    Q1 = rfm[i].quantile(0.5)
    Q3 = rfm[i].quantile(0.95)
    IQR = Q3 - Q1
    
    
    lower_limit = rfm[i].quantile(0.5) - (IQR * 1.5)
    lower_limit_extreme = rfm[i].quantile(0.5) - (IQR * 3)
    upper_limit = rfm[i].quantile(0.95) + (IQR * 1.5)
    upper_limit_extreme = rfm[i].quantile(0.5) + (IQR * 3)
    print('Lower Limit:', lower_limit)
    print('Lower Limit Extreme:', lower_limit_extreme)
    print('Upper Limit:', upper_limit)
    print('Upper Limit Extreme:', upper_limit_extreme)

def percent_outliers(i):
    Q1 = rfm[i].quantile(0.5)
    Q3 = rfm[i].quantile(0.95)
    IQR = Q3 - Q1
    
    lower_limit = rfm[i].quantile(0.5) - (IQR * 1.5)
    lower_limit_extreme = rfm[i].quantile(0.5) - (IQR * 3)
    upper_limit = rfm[i].quantile(0.95) + (IQR * 1.5)
    upper_limit_extreme = rfm[i].quantile(0.95) + (IQR * 3)
    print('Lower Limit: {} %'.format(rfm[(rfm[i] >= lower_limit)].shape[0]/ rfm.shape[0]*100))
    print('Lower Limit Extereme: {} %'.format(rfm[(rfm[i] >= lower_limit_extreme)].shape[0]/rfm.shape[0]*100))
    print('Upper Limit: {} %'.format(rfm[(rfm[i] >= upper_limit)].shape[0]/ rfm.shape[0]*100))
    print('Upper Limit Extereme: {} %'.format(rfm[(rfm[i] >= upper_limit_extreme)].shape[0]/rfm.shape[0]*100))

In [ ]:
sns.boxplot(x=rfm["Recency"])

In [ ]:
sns.boxplot(x=rfm["Frequency"])

La columna Frecuencia no elimina los valores atípicos porque los valores atípicos aquí son de hecho la posibilidad de que los clientes realicen muchas transacciones porque estos datos se toman de 2017-2018

In [ ]:
sns.boxplot(x=rfm["Monetary"])

In [ ]:
print(limit('Monetary'))
print('-'*50)
print(percent_outliers('Monetary'))

In [ ]:
outliers1_drop = rfm[(rfm['Monetary'] > 1500)].index
rfm.drop(outliers1_drop, inplace=True)

## ** 3.3 Percentil RFM **

En esta etapa, se crearán grupos de clientes basados ​​en Recency, Frequency y Monetary dividiéndolos en grupos de 3 **** en cada uno.

In [ ]:
# Crear grupos de clientes basados en Recency, Frequency y Monetary
#Porque Recency si cuantos menos días mejor, hará el pedido al revés
r_labels = range(3, 0, -1)
r_groups = pd.qcut(rfm.Recency, q = 3, labels = r_labels).astype('int')

# Debido a que la frecuencia está muy en el valor 1, entonces no puede usar qcut,
#porque el valor se apoyará más
f_groups = pd.qcut(rfm.Frequency.rank(method='first'), 3).astype('str')
#rfm['F'] = np.where((rfm['Frequency'] != 1) & (rfm['Frequency'] != 2), 3, rfm.Frequency)

m_labels = range(1, 4)
m_groups = pd.qcut(rfm.Monetary, q = 3, labels = m_labels).astype('int')

In [ ]:
rfm['R'] = r_groups.values
rfm['F'] = f_groups.values
rfm['M'] = m_groups.values

In [ ]:
rfm.tail(100)

In [ ]:
rfm['F'].value_counts()

In [ ]:
rfm['F'] = rfm['F'].replace({'(0.999, 30871.333]' : 1,
                             '(30871.333, 61741.667]' : 2,
                             '(61741.667, 92612.0]' : 3}).astype('int')

In [ ]:
rfm['RFM_Segment'] = rfm.apply(lambda x: str(x['R']) + str(x['F']) + str(x['M']), axis = 1)
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].sum(axis = 1)
rfm.head()

In [ ]:
score_labels = ['Bronze', 'Silver', 'Gold']
score_groups = pd.qcut(rfm.RFM_Score, q=3, labels = score_labels)
rfm['RFM_Level'] = score_groups.values
rfm.head()

## ** 3.4 Manejo de la inclinación y el escalado **

Para los datos que ingresarán al algoritmo K-Means, la distribución es normal porque facilitará la formación de grupos, luego no olvides igualar la escala de cada columna porque se medirá en función de la distancia entre cada entrada.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
plt.subplot(3, 1, 1); sns.distplot(rfm.Recency, label = 'Recency')
plt.subplot(3, 1, 2); sns.distplot(rfm['Frequency'], kde_kws={'bw': 0.1}, label='Frequency')
plt.subplot(3, 1, 3); sns.distplot(rfm.Monetary, label = 'Monetary')

plt.tight_layout()
plt.show()

In [ ]:
from scipy import stats

rfm_log = rfm[['Recency', 'Monetary']].apply(np.log, axis = 1).round(3)
rfm_log['Frequency'] = stats.boxcox(rfm['Frequency'])[0]
rfm_log.head()

In [ ]:
scaler = StandardScaler()
minmax = MinMaxScaler()
robust = RobustScaler()
rfm_scaled = scaler.fit_transform(rfm_log)

In [ ]:
rfm_scaled = pd.DataFrame(rfm_scaled, index = rfm.index, columns = rfm_log.columns)
rfm_scaled.head()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
plt.subplot(3, 1, 1); sns.distplot(rfm_scaled.Recency, label = 'Recency')
plt.subplot(3, 1, 2); sns.distplot(rfm_scaled.Frequency, kde_kws={'bw': 0.1}, label='Frequency')
plt.subplot(3, 1, 3); sns.distplot(rfm_scaled.Monetary, label = 'Monetary')

plt.tight_layout()
plt.show()

## ** 3.5 Agrupamiento de K-Means **

Aquí la agrupación se realizará utilizando el algoritmo K-Means para obtener el grupo óptimo de datos RFM

In [ ]:
wcss = {}

for i in range(1, 11):
    kmeans = KMeans(n_clusters= i, init= 'k-means++', max_iter= 300)
    kmeans.fit(rfm_scaled)
    wcss[i] = kmeans.inertia_
    
fig, ax = plt.subplots(figsize=(16, 9))
sns.pointplot(x = list(wcss.keys()), y = list(wcss.values()))
plt.title('Elbow Method')
plt.xlabel('K Numbers')
plt.ylabel('WCSS')
plt.show()

In [ ]:
clus = KMeans(n_clusters=4, n_init=10, init= 'k-means++', max_iter= 300)
clus.fit(rfm_scaled)

In [ ]:
rfm_scaled.shape

In [ ]:
clus.labels_

In [ ]:
rfm['K_Cluster'] = clus.labels_
rfm.head()

In [ ]:
all_data_cluster=all_data.merge(rfm['K_Cluster'],left_on='customer_unique_id',right_index=True)

In [ ]:
for ncluster in all_data_cluster['K_Cluster'].unique():
    print('Num Cluster: %s' % ncluster)
    print(all_data_cluster[all_data_cluster['K_Cluster']==ncluster].describe())

## ** 3.6 Evaluación **

Después de crear un modelo usando KMeans, ahora evaluaremos el modelo si los grupos que forma son realmente diferentes de cada grupo usando el Análisis de Silhouuette. Cuanto más se acerca al número 1, la diferencia que tiene con otras computadoras es cada vez más clara, si el valor está cerca de 0, entonces se parece cada vez más a un grupo cercano y, si -1, hizo un grupo incorrecto.

In [ ]:
visualizer = SilhouetteVisualizer(clus)

visualizer.fit(rfm_scaled) 
visualizer.poof() 

## ** 3.5 Visualización **

Visualizaremos para que sea más fácil ver el clúster que creamos con el clúster del algoritmo KMeans. Usaremos 'Snake Plot' en el mundo del marketing para ver qué categorías se incluyen en un clúster.



In [ ]:
rfm_scaled['K_Cluster'] = clus.labels_
rfm_scaled['RFM_Level'] = rfm.RFM_Level
rfm_scaled.reset_index(inplace = True)
rfm_scaled.head()

In [ ]:
rfm_melted = pd.melt(frame= rfm_scaled, id_vars= ['customer_unique_id', 'RFM_Level', 'K_Cluster'], 
                     var_name = 'Metrics', value_name = 'Value')
rfm_melted.head()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
sns.lineplot(x = 'Metrics', y = 'Value', hue = 'RFM_Level', data = rfm_melted)
plt.title('Snake Plot of RFM')
plt.legend(loc = 'upper right')

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
sns.lineplot(x = 'Metrics', y = 'Value', hue = 'K_Cluster', data = rfm_melted)
plt.title('Snake Plot of K_cluster')
plt.legend(loc = 'upper right')

Como podemos ver en los dos gráficos anteriores, podemos concluir con base en grupos:
1. Bronce (1 y 0): la frecuencia no es demasiado alta en comparación con la transacción nominal, pero la última vez que realizó una transacción rápida
2. Plata (2): la frecuencia es bastante alta y la transacción nominal es bastante alta, pero la última vez que realizó una transacción fue bastante larga
3. Oro (3): la frecuencia de gasto es alta y la cantidad gastada también es alta, pero el tiempo de transacción es largo

In [ ]:
# ¿Cuántos clientes hay según su categoría?
rfm_cus_level = rfm_scaled.groupby('K_Cluster')['customer_unique_id'].nunique().reset_index()


fig=plt.figure(figsize=(16,9))
sns.barplot(y=rfm_cus_level['customer_unique_id'], x=rfm_cus_level['K_Cluster'], palette="Greens_d")
plt.title('Customer Based on RFM Level',fontsize=20)
plt.xlabel('RFMLevel',fontsize=17)
plt.ylabel('Amount of Customer',fontsize=17)

# ** 4. CONCLUSIÓN**

En cuanto a los objetivos de este proyecto, que es facilitar que el departamento de marketing lleve a cabo campañas o promociones basadas en la segmentación de clientes, estas son las conclusiones:

1. Bronce: un cliente que no realiza compras con demasiada frecuencia y el valor nominal de la transacción es bajo, pero la última vez que realizó una transacción rápida. Hay 36,000 clientes de este tipo.
    - Acción: puede probarse dando descuentos u ofertas con un precio nominal asequible para que la tasa de conversión aumente porque el número de clientes de la categoría Bronce es bastante
2. Plata: Clientes que con frecuencia realizan compras y la transacción nominal es bastante alta, pero la última vez que realiza transacciones es bastante larga. Hay 42,000 clientes de este tipo.
    - Acción: dada una combinación de descuentos y campañas después de la transacción para aumentar las compras mediante el uso de un correo electrónico personalizado que puede dar un toque personal.
3. Oro: Clientes que a menudo compran y también realizan muchas transacciones nominales, pero el último tiempo de transacción es largo. Hay 15000 clientes de este tipo.
    - Acción: se suele dar una campaña después de realizar una transacción para volver a realizar una compra. También podría ser recompensado porque con mayor frecuencia realizan transacciones y el valor nominal es alto.

